In [288]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import warnings
warnings.filterwarnings('ignore')


In [289]:
data = pd.read_csv('data/spam_data_raw.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [290]:
train_data = data[0: -1000]
test_data = data[-1000:]

#spam_df = train_data.loc[train_data['Category'] == 'spam']
#ham_df = train_data.loc[train_data['Category'] == 'ham']

<h3 style='color:blue'>Первый этап: обработка исходного текста</h3>

<h4>Обработка текста. Я выделил несколько категорий, которые может содержать потенциальный спам:</h4>
<ul>
    <li>Ссылка на веб-сайт</li>
    <li>Номер телефона</li>
    <li>Названия гаджетов от apple</li>
    <li>"Маркетинговые" слова. Например: free, chance, action</li>
    <li>Слова, обозначающие характеристику девайса. Пока что единственное слово - gb</li>
    <li>Слэш, означающий стоимость за определенный период. Например: 150p/week</li>
</ul>

In [291]:
# Ф-ия принимает строку и превращает ее в список слов, заменяя при этом сигнальне слова на названия категории 
def replace_special_words_with_categories(msg):
    
    def is_phone_number(wrd):
        digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        for j in digits:
            if wrd.find(j) != -1 and len(wrd) >= 11 and wrd[0] == '0':
                return True
            
    def is_market_word(wrd):
        market_wrds = ['free', 'chance', 'action']
        for j in market_wrds:
            if wrd.find(j) != -1:
                return True
                    
    def is_website(wrd):
        if wrd.find('http') == 0 or wrd.find('www') == 0 or len(wrd) - wrd.find('com') == 3:
            return True
        elif (wrd.find('.uk') != -1) or (wrd.find('.com') != -1) or (wrd.find('www.') != -1) or (wrd.find('http') != -1):
            return True
    
    def is_apple_brand(wrd):
        apple_devices = ['ipod', 'iphone', 'ipad']
        for j in apple_devices:
            if wrd.find(j) != -1:
                return True
    def is_gigabyte(wrd):
        gigabyte = 'gb'
        if (len(wrd) - wrd.find(gigabyte)) == 2:
            return True
    
    def is_cost_per_period(wrd):
        if wrd.find('p/') != -1:
            return True
    
    def is_age(wrd):
        return
    
    msg = msg.lower()
    wordlist = msg.split()
    
    for word in wordlist:
        if is_phone_number(word):
            wordlist[wordlist.index(word)] = 'phonenumber'
        elif is_market_word(word):
            wordlist[wordlist.index(word)] = 'marketword'
        elif is_website(word) and len(word)>3:
            wordlist[wordlist.index(word)] = 'website'
        elif is_apple_brand(word):
            wordlist[wordlist.index(word)] = 'appledevice'
        elif is_gigabyte(word) and len(word) > 2:
            wordlist[wordlist.index(word)] = 'marketword'
        elif is_cost_per_period(word):
            wordlist[wordlist.index(word)] = 'costperperiod'
            
        
    return wordlist

<h4> Преобразование сообщений</h4>

In [292]:
# Обновленные датасеты с преобразованными сообщениями
train_data['Message'] = train_data['Message'].apply(lambda row: replace_special_words_with_categories(row))
spam_df = train_data.loc[train_data['Category'] == 'spam']
ham_df = train_data.loc[train_data['Category'] == 'ham']

# Сохраним новый датасет в csv-формате
train_data.to_csv("data/preprocessed_train_data.csv")

<h4>Посчитаем среднюю встречаемость стоп-слов для каждого класса</h4>

In [293]:
# Ф-ия подсчета индикаторных слов в сообщении
def is_ind_words(row, retlist):
    ind_wrds = ['phonenumber', 'marketword', 'website', 'appledevice', 'gigabyte', 'costperperiod']
    count = 0
    for word in row:
        if word in ind_wrds:
            count += 1
    retlist.append(count)    
    
# Подсчет общего кол-ва слов в спамах
retlist = []
spam_df['Message'].apply(lambda row: is_ind_words(row, retlist))

# Подсчет общего кол-ва слов в норм.сообщениях
hamretlist = []
ham_df['Message'].apply(lambda row: is_ind_words(row, hamretlist))

print('Средняя встречаемость ключевых слов в одном спаме:', sum(retlist)/len(spam_df))
print('Средняя встречаемость ключевых слов в одном нормальном сообщении:', sum(hamretlist)/len(ham_df))

Средняя встречаемость ключевых слов в одном спаме: 1.268729641693811
Средняя встречаемость ключевых слов в одном нормальном сообщении: 0.024002021222839818


<p>В среднем, в ста спамовых сообщениях содержится 127 ключевых слов</p>
<p>В то же время, в ста нормальных сообщениях лишь 2 раза встречаются ключевые слова</p>
<p>Значит, закономерность найдена. По крайней мере для данной выборки</p>

<h4>Теперь посчитаем среднюю встречаемость слов на тестовых данных</h4>

In [294]:
test_data['Message'] = test_data['Message'].apply(lambda row: replace_special_words_with_categories(row))
test_spam_df = test_data.loc[test_data['Category'] == 'spam']
test_ham_df = test_data.loc[test_data['Category'] == 'ham']

In [295]:
# Подсчет общего кол-ва слов в спамах
retlist = []
test_spam_df['Message'].apply(lambda row: is_ind_words(row, retlist))

# Подсчет общего кол-ва слов в норм.сообщениях
hamretlist = []
test_ham_df['Message'].apply(lambda row: is_ind_words(row, hamretlist))

print('Средняя встречаемость ключевых слов в одном спаме:', sum(retlist)/len(test_spam_df))
print('Средняя встречаемость ключевых слов в одном нормальном сообщении:', sum(hamretlist)/len(test_ham_df))

Средняя встречаемость ключевых слов в одном спаме: 1.368421052631579
Средняя встречаемость ключевых слов в одном нормальном сообщении: 0.01845444059976932


На тестовой выборе результаты еще лучше. Закономерность в данных найдена хорошо

<h3 style='color:blue'>Второй этап: создание нового датасета</h3>

<p>Исходные сообщения обработаны. Следующая задача: создание датасета встречаемости слов в сообщениях</p>

<h4>Удаление стоп-слов из сообщений и составление списка уникальных слов(будущих признаков)</h4>

In [296]:
# special chars to be removed
spec_chars = string.punctuation + '\n\xa0«»\t—…\r€’”“'
# function to remove special chars
def remove_chars_from_text(text, chars):
    return "".join([ch for ch in text if ch not in chars])

def preprocessing_msg(msg):
    msg = remove_chars_from_text(msg, spec_chars).lower()
    msg = remove_chars_from_text(msg, string.digits)
    return msg

def create_unique_words_list(msg, wordlist):
    for word in range(len(msg)):
        msg[word] = preprocessing_msg(msg[word])
        if msg[word] not in wordlist:
            wordlist.append(msg[word])
    return wordlist


In [297]:
def from_column_to_unique_words_list(column):
    '''Ф-ия для составления признаков(уникальных слов длины больше 2 и очищенных от стоп-слов)''' 
    wordlist = []
    column.apply(lambda row: create_unique_words_list(row, wordlist))
    # Очистка от стоп-слов
    #from nltk.corpus import stopwords
    #nltk.download('stopwords')
    #stopwords = stopwords.words("english")
    #filtered_words = [word for word in wordlist if word not in stopwords]
    # Удаление слов, в которых меньше 3 символов
    #idx_2_del = []
    #for word in range(0, len(filtered_words)):
        #if len(filtered_words[word]) < 3:
            #idx_2_del.append(word)
    #filtered_words[:] = [x for i,x in enumerate(filtered_words) if i not in idx_2_del]      
    return wordlist

def from_column_to_words_list(row):
    ''' Ф-ия удаляет из сообщения стоп-слова и слова с длиной меньше 3 букв. Возвращает новое сообщение '''
    for i in range(len(row)):
        row[i] = preprocessing_msg(row[i])
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    stopwords = stopwords.words("english")
    filtered_words = [word for word in row if word not in stopwords]
    idx_2_del = []
    for word in range(0, len(filtered_words)):
        if len(filtered_words[word]) < 3:
            idx_2_del.append(word)
    filtered_words[:] = [x for i,x in enumerate(filtered_words) if i not in idx_2_del]      
    return filtered_words

In [298]:
# Приведение каждого сообщения к аналогичному виду(без стоп-слов и слов где менее 3 букв), только слова не уникальные
train_data['Message'] = train_data['Message'].apply(lambda row: from_column_to_words_list(row))


# Список уникальных лов в преобразованных сообщениях
unq_msgs = []
train_data['Message'].apply(lambda row: create_unique_words_list(row, unq_msgs))
print('Кол-во уникальных слов в преобразованных сообщениях: ', len(unq_msgs))

# Список уникальных слов во всех сообщениях
unique_words = from_column_to_unique_words_list(train_data['Message'])
print('Кол-во уникальных слов во всех сообщениях:', len(unique_words))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

...

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is alrea

Кол-во уникальных слов в преобразованных сообщениях:  7324
Кол-во уникальных слов во всех сообщениях: 7324


<h4>Составление датасета</h4>

In [299]:
%%time
# Составлене датасета встречаемости слов
unique_words = sorted(unique_words)
for col in range(len(unique_words)):
    train_data[unique_words[col]] = 0
    
train_data.head(3)

Wall time: 12.6 s


,Category,Message,aah,aaniye,aaooooright,aathilove,aathiwhere,abbey,abeg,abelu,...,£minmobsmorelkpoboxhpfl,£month,£morefrmmob,£msg,£perwksub,£pm,£pmmorefrommobilebremovedmobypoboxlsyf,£week,£wk,üll
0,ham,"[jurong, point, crazy, available, bugis, great...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[lar, joking, wif, oni]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,spam,"[marketword, entry, wkly, comp, win, cup, fina...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Заполнение датасета </h4>

In [300]:
%%time
# Подсчет встречаемости по строкам

def words_counter(idx, row) -> None:
    for word in row:
        train_data.at[idx,word] += 1
    print(f'{idx} сообщение обработано')
for idx, row in enumerate(train_data['Message']):
    words_counter(idx, row)
#train_data['Message'] = train_data['Message'].apply(lambda row: words_counter(enumerate(row)))

0 сообщение обработано
1 сообщение обработано
2 сообщение обработано
3 сообщение обработано
4 сообщение обработано
5 сообщение обработано
6 сообщение обработано
7 сообщение обработано
8 сообщение обработано
9 сообщение обработано
10 сообщение обработано
11 сообщение обработано
12 сообщение обработано
13 сообщение обработано
14 сообщение обработано
15 сообщение обработано
16 сообщение обработано
17 сообщение обработано
18 сообщение обработано
19 сообщение обработано
20 сообщение обработано
21 сообщение обработано
22 сообщение обработано
23 сообщение обработано
24 сообщение обработано
25 сообщение обработано
26 сообщение обработано
27 сообщение обработано
28 сообщение обработано
29 сообщение обработано
30 сообщение обработано
31 сообщение обработано
32 сообщение обработано
33 сообщение обработано
34 сообщение обработано
35 сообщение обработано
36 сообщение обработано
37 сообщение обработано
38 сообщение обработано
39 сообщение обработано
40 сообщение обработано
41 сообщение обработано
42

429 сообщение обработано
430 сообщение обработано
431 сообщение обработано
432 сообщение обработано
433 сообщение обработано
434 сообщение обработано
435 сообщение обработано
436 сообщение обработано
437 сообщение обработано
438 сообщение обработано
439 сообщение обработано
440 сообщение обработано
441 сообщение обработано
442 сообщение обработано
443 сообщение обработано
444 сообщение обработано
445 сообщение обработано
446 сообщение обработано
447 сообщение обработано
448 сообщение обработано
449 сообщение обработано
450 сообщение обработано
451 сообщение обработано
452 сообщение обработано
453 сообщение обработано
454 сообщение обработано
455 сообщение обработано
456 сообщение обработано
457 сообщение обработано
458 сообщение обработано
459 сообщение обработано
460 сообщение обработано
461 сообщение обработано
462 сообщение обработано
463 сообщение обработано
464 сообщение обработано
465 сообщение обработано
466 сообщение обработано
467 сообщение обработано
468 сообщение обработано


1021 сообщение обработано
1022 сообщение обработано
1023 сообщение обработано
1024 сообщение обработано
1025 сообщение обработано
1026 сообщение обработано
1027 сообщение обработано
1028 сообщение обработано
1029 сообщение обработано
1030 сообщение обработано
1031 сообщение обработано
1032 сообщение обработано
1033 сообщение обработано
1034 сообщение обработано
1035 сообщение обработано
1036 сообщение обработано
1037 сообщение обработано
1038 сообщение обработано
1039 сообщение обработано
1040 сообщение обработано
1041 сообщение обработано
1042 сообщение обработано
1043 сообщение обработано
1044 сообщение обработано
1045 сообщение обработано
1046 сообщение обработано
1047 сообщение обработано
1048 сообщение обработано
1049 сообщение обработано
1050 сообщение обработано
1051 сообщение обработано
1052 сообщение обработано
1053 сообщение обработано
1054 сообщение обработано
1055 сообщение обработано
1056 сообщение обработано
1057 сообщение обработано
1058 сообщение обработано
1059 сообщен

1569 сообщение обработано
1570 сообщение обработано
1571 сообщение обработано
1572 сообщение обработано
1573 сообщение обработано
1574 сообщение обработано
1575 сообщение обработано
1576 сообщение обработано
1577 сообщение обработано
1578 сообщение обработано
1579 сообщение обработано
1580 сообщение обработано
1581 сообщение обработано
1582 сообщение обработано
1583 сообщение обработано
1584 сообщение обработано
1585 сообщение обработано
1586 сообщение обработано
1587 сообщение обработано
1588 сообщение обработано
1589 сообщение обработано
1590 сообщение обработано
1591 сообщение обработано
1592 сообщение обработано
1593 сообщение обработано
1594 сообщение обработано
1595 сообщение обработано
1596 сообщение обработано
1597 сообщение обработано
1598 сообщение обработано
1599 сообщение обработано
1600 сообщение обработано
1601 сообщение обработано
1602 сообщение обработано
1603 сообщение обработано
1604 сообщение обработано
1605 сообщение обработано
1606 сообщение обработано
1607 сообщен

2154 сообщение обработано
2155 сообщение обработано
2156 сообщение обработано
2157 сообщение обработано
2158 сообщение обработано
2159 сообщение обработано
2160 сообщение обработано
2161 сообщение обработано
2162 сообщение обработано
2163 сообщение обработано
2164 сообщение обработано
2165 сообщение обработано
2166 сообщение обработано
2167 сообщение обработано
2168 сообщение обработано
2169 сообщение обработано
2170 сообщение обработано
2171 сообщение обработано
2172 сообщение обработано
2173 сообщение обработано
2174 сообщение обработано
2175 сообщение обработано
2176 сообщение обработано
2177 сообщение обработано
2178 сообщение обработано
2179 сообщение обработано
2180 сообщение обработано
2181 сообщение обработано
2182 сообщение обработано
2183 сообщение обработано
2184 сообщение обработано
2185 сообщение обработано
2186 сообщение обработано
2187 сообщение обработано
2188 сообщение обработано
2189 сообщение обработано
2190 сообщение обработано
2191 сообщение обработано
2192 сообщен

2541 сообщение обработано
2542 сообщение обработано
2543 сообщение обработано
2544 сообщение обработано
2545 сообщение обработано
2546 сообщение обработано
2547 сообщение обработано
2548 сообщение обработано
2549 сообщение обработано
2550 сообщение обработано
2551 сообщение обработано
2552 сообщение обработано
2553 сообщение обработано
2554 сообщение обработано
2555 сообщение обработано
2556 сообщение обработано
2557 сообщение обработано
2558 сообщение обработано
2559 сообщение обработано
2560 сообщение обработано
2561 сообщение обработано
2562 сообщение обработано
2563 сообщение обработано
2564 сообщение обработано
2565 сообщение обработано
2566 сообщение обработано
2567 сообщение обработано
2568 сообщение обработано
2569 сообщение обработано
2570 сообщение обработано
2571 сообщение обработано
2572 сообщение обработано
2573 сообщение обработано
2574 сообщение обработано
2575 сообщение обработано
2576 сообщение обработано
2577 сообщение обработано
2578 сообщение обработано
2579 сообщен

2904 сообщение обработано
2905 сообщение обработано
2906 сообщение обработано
2907 сообщение обработано
2908 сообщение обработано
2909 сообщение обработано
2910 сообщение обработано
2911 сообщение обработано
2912 сообщение обработано
2913 сообщение обработано
2914 сообщение обработано
2915 сообщение обработано
2916 сообщение обработано
2917 сообщение обработано
2918 сообщение обработано
2919 сообщение обработано
2920 сообщение обработано
2921 сообщение обработано
2922 сообщение обработано
2923 сообщение обработано
2924 сообщение обработано
2925 сообщение обработано
2926 сообщение обработано
2927 сообщение обработано
2928 сообщение обработано
2929 сообщение обработано
2930 сообщение обработано
2931 сообщение обработано
2932 сообщение обработано
2933 сообщение обработано
2934 сообщение обработано
2935 сообщение обработано
2936 сообщение обработано
2937 сообщение обработано
2938 сообщение обработано
2939 сообщение обработано
2940 сообщение обработано
2941 сообщение обработано
2942 сообщен

3485 сообщение обработано
3486 сообщение обработано
3487 сообщение обработано
3488 сообщение обработано
3489 сообщение обработано
3490 сообщение обработано
3491 сообщение обработано
3492 сообщение обработано
3493 сообщение обработано
3494 сообщение обработано
3495 сообщение обработано
3496 сообщение обработано
3497 сообщение обработано
3498 сообщение обработано
3499 сообщение обработано
3500 сообщение обработано
3501 сообщение обработано
3502 сообщение обработано
3503 сообщение обработано
3504 сообщение обработано
3505 сообщение обработано
3506 сообщение обработано
3507 сообщение обработано
3508 сообщение обработано
3509 сообщение обработано
3510 сообщение обработано
3511 сообщение обработано
3512 сообщение обработано
3513 сообщение обработано
3514 сообщение обработано
3515 сообщение обработано
3516 сообщение обработано
3517 сообщение обработано
3518 сообщение обработано
3519 сообщение обработано
3520 сообщение обработано
3521 сообщение обработано
3522 сообщение обработано
3523 сообщен

4344 сообщение обработано
4345 сообщение обработано
4346 сообщение обработано
4347 сообщение обработано
4348 сообщение обработано
4349 сообщение обработано
4350 сообщение обработано
4351 сообщение обработано
4352 сообщение обработано
4353 сообщение обработано
4354 сообщение обработано
4355 сообщение обработано
4356 сообщение обработано
4357 сообщение обработано
4358 сообщение обработано
4359 сообщение обработано
4360 сообщение обработано
4361 сообщение обработано
4362 сообщение обработано
4363 сообщение обработано
4364 сообщение обработано
4365 сообщение обработано
4366 сообщение обработано
4367 сообщение обработано
4368 сообщение обработано
4369 сообщение обработано
4370 сообщение обработано
4371 сообщение обработано
4372 сообщение обработано
4373 сообщение обработано
4374 сообщение обработано
4375 сообщение обработано
4376 сообщение обработано
4377 сообщение обработано
4378 сообщение обработано
4379 сообщение обработано
4380 сообщение обработано
4381 сообщение обработано
4382 сообщен

<h3 style='color:blue'>Третий этап: построение модели</h3>

<h3>Модель: наивный Байес</h3>
<h5>ham - класс 0. spam - класс 1</h5>

<p>В сформированном датасете значение x[i][j] показывает: сколько раз слово j встретилось в сообщении x[i]</p>
<p>Так как значением j-го признака является количество вхождений в  i-й документ, это наталкивает на мысль, что каждый признак(слово) имеет Пуассоновское или мультиномиальное распределение</p>

<h4 style="color:red">Предположение 1: признаки распределены по Пуассону</h4>
    <p>Предположим, что признаки распределены по закону Пуассона</p>
    <p>Тогда при подстановке плотности Пуассоновского распределения в формулу наивного Байеса получим:</p>

In [301]:
df = train_data.drop('Message', axis=1)
ham_df = df.loc[df['Category'] == 'ham']
spam_df = df.loc[df['Category'] == 'spam']

In [302]:
%%time

# Обучение наивного байеса
# Словарь средрей встречаемости каждого слова в сообщениях для каждого класса
mean_vals = {'ham': [],
             'spam': []
            }

for i in range(1, len(df.columns)-1):
    mean_vals['ham'].append(ham_df[df.columns[i]].mean())
for i in range(1, len(df.columns)-1):
    mean_vals['spam'].append(spam_df[df.columns[i]].mean())

# Средняя длина сообщения для каждого класса
N_ham = sum(mean_vals['ham'])
N_spam = sum(mean_vals['spam'])

# Априорные вероятности классов
P_ham = len(ham_df)/len(df)
P_spam = len(spam_df)/len(df)

# Тета для каждого класса для каждого столбца
thetas_dict = {}
thetas_dict['ham'] = np.log(mean_vals['ham'])
thetas_dict['spam'] = np.log(mean_vals['spam'])
# Замена минус бесконечностей на конечное число
thetas_dict['ham'] = [-999999 if x == -np.inf else x for x in thetas_dict['ham']]
thetas_dict['spam'] = [-999999 if x == -np.inf else x for x in thetas_dict['spam']]

Wall time: 1.53 s


Предсказания:

In [330]:
%%time
def predict(df, thetas_dict, P_ham, P_spam, N_ham, N_spam, lambda_ham=1, lambda_spam=1):
    
    preds = np.ones((len(df),))
    preds_proba = []
    # Для каждого объекта
    for row in range(len(df)):
        # Для каждого столбца
        ax = []
        sum_ham = 0
        sum_spam = 0
        sum_list_ham = []
        sum_list_spam = []
        # Для ham
        for col in range(1, len(df.columns)-1):
            ans = thetas_dict['ham'][col-1] * df.iloc[row,col] + np.log(lambda_ham*P_ham) - N_ham
            sum_ham += ans
            sum_list_ham.append(ans)
        # Для spam
        for col in range(1, len(df.columns)-1):
            ans = thetas_dict['spam'][col-1] * df.iloc[row, col] + np.log(lambda_spam*P_spam) - N_spam
            sum_spam += ans
            sum_list_spam.append(ans)
            
        ax = [sum_ham, sum_spam]
        preds[row] = np.argmax(ax)
        preds_proba.append(ax)
        print(f'{row} объект классифицирован. proba = {ax}, класс: {np.argmax(ax)}')
    return preds, preds_proba
        
        

Wall time: 0 ns


In [ ]:
%%time
# Прогноз на обуч.выборке
preds, preds_proba = predict(df, thetas_dict, P_ham, P_spam, N_ham, N_spam) 

Преобразуем в пандасовский формат

In [ ]:
preds = pd.Series(data=preds)
preds_proba = pd.DataFrame(np.array(preds_proba))

In [ ]:
train_data = train_data[['Message' ,'Category']]

In [ ]:
train_data['Pred'] = preds
train_data['Proba_ham'] = preds_proba[0]
train_data['Proba_spam'] = preds_proba[1]

In [ ]:
train_data['Pred'] = train_data['Pred'].apply(lambda x: 'ham' if (x == 0.0) else 'spam')

<h4>Итоговая таблица с предсказаниями на обучающей выборке</h4>

In [ ]:
train_data

In [305]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
print(classification_report(train_data['Category'], train_data['Pred']))

<h4>На тестовой выборке</h4>

In [306]:
%%time
'''Сначала приведение тестового датасета к нужному виду'''
# Замена исходного текста на список слов
test_data['Message'] = test_data['Message'].apply(lambda row: from_column_to_words_list(row))

# Составлене датасета встречаемости слов
for col in range(len(unique_words)):
    test_data[unique_words[col]] = 0
    
# индексы в test_data нумеруются не с 0, а с 4752. Надо привести индексы к нужному виду
old_idx = test_data.index
new_idx = range(0, len(test_data))
for i in range(len(old_idx)):
    test_data = test_data.rename(index={old_idx[i]: new_idx[i]})
    
# Заполнение датасета
def words_counter(idx, row) -> None:
    for word in row:
        if word in test_data.columns:
            test_data.at[idx,word] += 1
    print(f'{idx} сообщение обработано')
for idx, row in enumerate(test_data['Message']):
    words_counter(idx, row)

# Удаление столбца с текстом сообщения
test_df = test_data.drop('Message', axis=1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

0 сообщение обработано
1 сообщение обработано
2 сообщение обработано
3 сообщение обработано
4 сообщение обработано
5 сообщение обработано
6 сообщение обработано
7 сообщение обработано
8 сообщение обработано
9 сообщение обработано
10 сообщение обработано
11 сообщение обработано
12 сообщение обработано
13 сообщение обработано
14 сообщение обработано
15 сообщение обработано
16 сообщение обработано
17 сообщение обработано
18 сообщение обработано
19 сообщение обработано
20 сообщение обработано
21 сообщение обработано
22 сообщение обработано
23 сообщение обработано
24 сообщение обработано
25 сообщение обработано
26 сообщение обработано
27 сообщение обработано
28 сообщение обработано
29 сообщение обработано
30 сообщение обработано
31 сообщение обработано
32 сообщение обработано
33 сообщение обработано
34 сообщение обработано
35 сообщение обработано
36 сообщение обработано
37 сообщение обработано
38 сообщение обработано
39 сообщение обработано
40 сообщение обработано
41 сообщение обработано
42

646 сообщение обработано
647 сообщение обработано
648 сообщение обработано
649 сообщение обработано
650 сообщение обработано
651 сообщение обработано
652 сообщение обработано
653 сообщение обработано
654 сообщение обработано
655 сообщение обработано
656 сообщение обработано
657 сообщение обработано
658 сообщение обработано
659 сообщение обработано
660 сообщение обработано
661 сообщение обработано
662 сообщение обработано
663 сообщение обработано
664 сообщение обработано
665 сообщение обработано
666 сообщение обработано
667 сообщение обработано
668 сообщение обработано
669 сообщение обработано
670 сообщение обработано
671 сообщение обработано
672 сообщение обработано
673 сообщение обработано
674 сообщение обработано
675 сообщение обработано
676 сообщение обработано
677 сообщение обработано
678 сообщение обработано
679 сообщение обработано
680 сообщение обработано
681 сообщение обработано
682 сообщение обработано
683 сообщение обработано
684 сообщение обработано
685 сообщение обработано


<p>Тестовый датасет приведен к нужному формату. Далее - предсказания модели:</p>

In [331]:
%%time
# Предсказания на тестовой выборке
preds, preds_proba = predict(test_df, thetas_dict, P_ham, P_spam, N_ham, N_spam) 

0 объект классифицирован. proba = [-54357.2646181234, -2113255.317888017], класс: 0
1 объект классифицирован. proba = [-54337.40252543006, -2113242.189129794], класс: 0
2 объект классифицирован. proba = [-2054340.1900169365, -113256.93500294887], класс: 1
3 объект классифицирован. proba = [-54320.80527069716, -113240.94718898193], класс: 0
4 объект классифицирован. proba = [-54346.64486990252, -4113236.947188407], класс: 0
5 объект классифицирован. proba = [-5054354.743028042, -113274.88851416405], класс: 1
6 объект классифицирован. proba = [-3054403.8778208205, -113305.34447544123], класс: 1
7 объект классифицирован. proba = [-7054359.691686949, -113283.67427638125], класс: 1
8 объект классифицирован. proba = [-54331.332004238204, -1113245.6740364986], класс: 0
9 объект классифицирован. proba = [-54348.99826970719, -2113257.4237628523], класс: 0
10 объект классифицирован. proba = [-54350.63458171646, -2113260.4268119405], класс: 0
11 объект классифицирован. proba = [-54364.21138580986

96 объект классифицирован. proba = [-54324.43480447316, -113243.30674089953], класс: 0
97 объект классифицирован. proba = [-54347.98860464922, -2113250.400884192], класс: 0
98 объект классифицирован. proba = [-54406.05254401341, -7113280.231974014], класс: 0
99 объект классифицирован. proba = [-54342.05936636679, -3113242.5523481467], класс: 0
100 объект классифицирован. proba = [-54350.941898695055, -3113246.985413951], класс: 0
101 объект классифицирован. proba = [-4054371.9155088305, -113291.13806360471], класс: 1
102 объект классифицирован. proba = [-54382.36825198404, -7113253.230238655], класс: 0
103 объект классифицирован. proba = [-54331.40611221035, -2113238.947188625], класс: 0
104 объект классифицирован. proba = [-1054376.1047564778, -2113273.45906809], класс: 0
105 объект классифицирован. proba = [-54328.77385450264, -113247.07179545188], класс: 0
106 объект классифицирован. proba = [-54320.80527069716, -113240.94718898193], класс: 0
107 объект классифицирован. proba = [-54

191 объект классифицирован. proba = [-54335.604535411076, -1113248.915977504], класс: 0
192 объект классифицирован. proba = [-54327.99015253465, -1113239.947188764], класс: 0
193 объект классифицирован. proba = [-54335.913784952274, -113250.20837603402], класс: 0
194 объект классифицирован. proba = [-54349.735280859335, -1113260.2121543603], класс: 0
195 объект классифицирован. proba = [-54342.25455569398, -2113244.2685714182], класс: 0
196 объект классифицирован. proba = [-2054358.5390689308, -113269.56199408488], класс: 1
197 объект классифицирован. proba = [-54339.4383696506, -2113248.8968069754], класс: 0
198 объект классифицирован. proba = [-54377.206857324294, -5113260.240874047], класс: 0
199 объект классифицирован. proba = [-54365.078619998494, -5113248.787178461], класс: 0
200 объект классифицирован. proba = [-54328.26041382425, -113251.01459011598], класс: 0
201 объект классифицирован. proba = [-54393.897185863956, -3113272.025668415], класс: 0
202 объект классифицирован. pro

286 объект классифицирован. proba = [-54344.841794339096, -113264.71389326939], класс: 0
287 объект классифицирован. proba = [-54320.80527069716, -113240.94718898193], класс: 0
288 объект классифицирован. proba = [-54394.42914437956, -8113255.335159385], класс: 0
289 объект классифицирован. proba = [-54389.52828110098, -5113273.509394036], класс: 0
290 объект классифицирован. proba = [-54332.551977383875, -113246.02076194019], класс: 0
291 объект классифицирован. proba = [-4054377.6680469355, -1113275.3109832811], класс: 1
292 объект классифицирован. proba = [-5054341.283855787, -113278.98016590856], класс: 1
293 объект классифицирован. proba = [-54345.55335703515, -1113262.4491145476], класс: 0
294 объект классифицирован. proba = [-54341.885504394835, -2113250.6885661534], класс: 0
295 объект классифицирован. proba = [-54413.218457286755, -9113273.772986842], класс: 0
296 объект классифицирован. proba = [-54333.97106156782, -1113246.3671836504], класс: 0
297 объект классифицирован. pr

380 объект классифицирован. proba = [-54385.05068391965, -6113266.10416555], класс: 0
381 объект классифицирован. proba = [-1054363.6767846, -2113253.889078478], класс: 0
382 объект классифицирован. proba = [-54380.74152553318, -4113263.921571833], класс: 0
383 объект классифицирован. proba = [-54360.67506981851, -6113244.696135874], класс: 0
384 объект классифицирован. proba = [-54349.51150304622, -3113248.707736954], класс: 0
385 объект классифицирован. proba = [-54371.210311108494, -3113275.812173287], класс: 0
386 объект классифицирован. proba = [-54331.20961509732, -113250.15958586986], класс: 0
387 объект классифицирован. proba = [-54332.71938198392, -2113239.750412492], класс: 0
388 объект классифицирован. proba = [-54350.31377348367, -4113249.0940311933], класс: 0
389 объект классифицирован. proba = [-54327.02232430711, -1113243.148307764], класс: 0
390 объект классифицирован. proba = [-54412.87261247703, -9113268.046139034], класс: 0
391 объект классифицирован. proba = [-54325

...

569 объект классифицирован. proba = [-4054389.7842090847, -113287.54101544767], класс: 1
570 объект классифицирован. proba = [-54386.05522930947, -5113268.748306623], класс: 0
571 объект классифицирован. proba = [-54359.93789110585, -3113260.4502008404], класс: 0
572 объект классифицирован. proba = [-54326.09303254976, -113247.36718391007], класс: 0
573 объект классифицирован. proba = [-54364.41895012023, -4113260.3245833227], класс: 0
574 объект классифицирован. proba = [-54389.27631345541, -5113265.792961164], класс: 0
575 объект классифицирован. proba = [-1054371.5447274742, -113277.4481107464], класс: 1
576 объект классифицирован. proba = [-54374.30017146594, -3113276.656389595], класс: 0
577 объект классифицирован. proba = [-54333.36159629539, -1113245.0909658477], класс: 0
578 объект классифицирован. proba = [-54358.002092850395, -2113264.5963378483], класс: 0
579 объект классифицирован. proba = [-1054349.1189299105, -1113275.1298482232], класс: 0
580 объект классифицирован. prob

663 объект классифицирован. proba = [-54352.79930774053, -3113248.9954188033], класс: 0
664 объект классифицирован. proba = [-54396.610250934835, -10113237.253391054], класс: 0
665 объект классифицирован. proba = [-3054372.2064290424, -113284.36270667678], класс: 1
666 объект классифицирован. proba = [-54339.48874837745, -1113255.8059762025], класс: 0
667 объект классифицирован. proba = [-54351.491354655096, -2113261.73679658], класс: 0
668 объект классифицирован. proba = [-54362.30490484231, -5113251.335972388], класс: 0
669 объект классифицирован. proba = [-54332.40244564991, -1113244.2877420646], класс: 0
670 объект классифицирован. proba = [-54336.031979425905, -3113237.947188627], класс: 0
671 объект классифицирован. proba = [-54405.884152876126, -9113257.857222594], класс: 0
672 объект классифицирован. proba = [-54344.30269550978, -1113257.5568877826], класс: 0
673 объект классифицирован. proba = [-54358.30858132852, -4113250.4233482904], класс: 0
674 объект классифицирован. prob

757 объект классифицирован. proba = [-54337.759200374, -1113250.222229145], класс: 0
758 объект классифицирован. proba = [-54332.489457026895, -2113238.947188656], класс: 0
759 объект классифицирован. proba = [-54361.17014387684, -3113253.2951504495], класс: 0
760 объект классифицирован. proba = [-54337.465952896084, -1113252.2601976693], класс: 0
761 объект классифицирован. proba = [-54382.85120539363, -8113245.09403125], класс: 0
762 объект классифицирован. proba = [-54344.96260589509, -3113237.9471884156], класс: 0
763 объект классифицирован. proba = [-54396.49030073087, -7113268.688015213], класс: 0
764 объект классифицирован. proba = [-54364.11233853376, -2113271.5806470993], класс: 0
765 объект классифицирован. proba = [-54335.83945873481, -1113252.7871785169], класс: 0
766 объект классифицирован. proba = [-54331.93417964055, -1113244.0645984672], класс: 0
767 объект классифицирован. proba = [-54337.67984471547, -2113243.1699593184], класс: 0
768 объект классифицирован. proba = [

851 объект классифицирован. proba = [-54333.88009227967, -1113252.4917900385], класс: 0
852 объект классифицирован. proba = [-54371.02664929224, -7113249.335972276], класс: 0
853 объект классифицирован. proba = [-54347.78122389228, -3113246.142787579], класс: 0
854 объект классифицирован. proba = [-54340.3376330818, -2113245.36718359], класс: 0
855 объект классифицирован. proba = [-1054370.5509754294, -1113282.1720826617], класс: 0
856 объект классифицирован. proba = [-54340.462796224754, -2113245.36718351], класс: 0
857 объект классифицирован. proba = [-54346.946197811965, -2113260.35050243], класс: 0
858 объект классифицирован. proba = [-54363.35190054668, -2113272.2396970945], класс: 0
859 объект классифицирован. proba = [-54333.634589331195, -1113246.3671838746], класс: 0
860 объект классифицирован. proba = [-54335.626514317795, -2113242.3714513117], класс: 0
861 объект классифицирован. proba = [-54341.89955977377, -2113251.0940313116], класс: 0
862 объект классифицирован. proba = 

945 объект классифицирован. proba = [-54429.25404079032, -11113270.856697228], класс: 0
946 объект классифицирован. proba = [-54382.077693728424, -6113263.7890667105], класс: 0
947 объект классифицирован. proba = [-54339.30109527169, -113257.08608676062], класс: 0
948 объект классифицирован. proba = [-54336.44540257233, -1113251.8939530968], класс: 0
949 объект классифицирован. proba = [-54334.0049631195, -1113245.6740364209], класс: 0
950 объект классифицирован. proba = [-54337.24032236651, -1113252.1596564574], класс: 0
951 объект классифицирован. proba = [-54334.467450330936, -113253.9156046268], класс: 0
952 объект классифицирован. proba = [-6054368.387394728, -113293.1620993087], класс: 1
953 объект классифицирован. proba = [-54371.9031774886, -5113255.658768974], класс: 0
954 объект классифицирован. proba = [-5054361.114372255, -113283.8425868749], класс: 1
955 объект классифицирован. proba = [-54353.57224176924, -3113249.4008841645], класс: 0
956 объект классифицирован. proba = 

<p>Наглядное сравнение истинных классов, предсказанных классов и оценок принадлежности к классам: </p>

In [332]:
preds = pd.Series(data=preds)
preds_proba = pd.DataFrame(np.array(preds_proba))
test_data = test_data[['Message' ,'Category']]
test_data['Pred'] = preds
test_data['Proba_ham'] = preds_proba[0]
test_data['Proba_spam'] = preds_proba[1]
test_data['Pred'] = test_data['Pred'].apply(lambda x: 'ham' if (x == 0.0) else 'spam')

In [333]:
test_data.sample(5)

,Message,Category,Pred,Proba_ham,Proba_spam
498,"[hai, dear, friends, new, amp, present, number...",ham,ham,-54363.724955,-4.113252e+06
906,"[probably, ltgt]",ham,ham,-54328.616837,-2.113239e+06
799,"[lol, got, friend, dog]",ham,ham,-54340.012570,-2.113248e+06
748,"[havent, got, topic, yet, rite]",ham,ham,-54338.827064,-1.113257e+06
176,"[put, sign, choose, number, pin, show, right]",ham,ham,-54360.652083,-1.132782e+05


Результат на тестовой выборке:

In [334]:
print(classification_report(test_data['Category'], test_data['Pred']))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       867
        spam       0.98      0.83      0.90       133

    accuracy                           0.98      1000
   macro avg       0.98      0.92      0.94      1000
weighted avg       0.98      0.98      0.98      1000



<h4>При предположении, что признаки распределены по Пуассоновскому распределению, получены следующие результаты:</h4>
<p>Accuracy: train - 1.00, test - 0.98</p>
<p>Выявлено спама: train - 98%, test - 83%</p>

<p>Accuracy изменилась совсем немного, однако процент выявленного спама упал на 15</p>
<p>Посмотрим на каких объектах ошибся алгоритм:</p>

In [337]:
# Объекты, на которых алгоритм ошибся
test_data.loc[test_data['Category'] != test_data['Pred']]

,Message,Category,Pred,Proba_ham,Proba_spam
104,"[babe, chloe, smashed, saturday, night, great,...",spam,ham,-1.054376e+06,-2.113273e+06
182,"[website, get, lots, cash, weekend, website, d...",spam,ham,-5.440407e+04,-1.133089e+05
188,"[thanks, continued, support, question, week, e...",spam,ham,-5.438377e+04,-1.113275e+06
226,"[santa, calling, would, little, ones, like, ca...",spam,ham,-1.054387e+06,-1.113291e+06
249,"[check, choose, babe, videos, smsshsexnetun, f...",spam,ham,-5.433639e+04,-1.132555e+05
334,"[warner, village, colin, farrell, swat, wkend,...",spam,ham,-1.054382e+06,-3.113269e+06
342,"[goal, arsenal, henry, liverpool, henry, score...",spam,ham,-3.054373e+06,-5.113267e+06
359,"[sexychat, girls, waiting, text, text, great, ...",spam,ham,-5.437764e+04,-1.132785e+05
377,"[amy, sending, marketword, phone, number, coup...",spam,ham,-5.437849e+04,-2.113272e+06
381,"[welcome, select, service, added, benefits, ca...",spam,ham,-1.054364e+06,-2.113254e+06


<p>Бросается в глаза, что в большинстве случаев оценки принадлежности к классам сравнимы: они одного порядка и не сильно отличаются</p>
<h5>Отсюда следует, что объекты пограничные. Можно решить проблему, добавив в формулу разные потери от ошибок в зависимости от класса. Сейчас потери от ошибок одинаковые и равны 1</h5>

<h4 style="color:red">Предположение 2: признаки распределены по мультиномиальному распределению</h4>

<p>Проработаем второй вариант: предположим, что признаки распределны по мультиномиальному закону</p>
<p>Если в формулу наивного байесовского классификатора подставить плотность мультиномиального распределения, то формула будет совсем немного отличаться от наивного байеса с пуассоновским распределением: свободный член Ny попадает под логарифм и домножается на новый параметр - длину конкретного документа</p>

In [338]:
%%time
def predict_multinomial(df, data_with_message, thetas_dict, P_ham, P_spam, N_ham, N_spam, lambda_ham=1, lambda_spam=1):
    
    preds = np.ones((len(df),))
    preds_proba = []
    # Для каждого объекта
    for row in range(len(df)):
        # Для каждого столбца
        ax = []
        sum_ham = 0
        sum_spam = 0
        sum_list_ham = []
        sum_list_spam = []
        # Для ham
        for col in range(1, len(df.columns)-1):
            ans = thetas_dict['ham'][col-1] * df.iloc[row,col] + np.log(lambda_ham * P_ham) - (len(data_with_message.iloc[row, 0]) * np.log(N_ham)) 
            sum_ham += ans
            sum_list_ham.append(ans)
        # Для spam
        for col in range(1, len(df.columns)-1):
            ans = thetas_dict['spam'][col-1] * df.iloc[row, col] + np.log(lambda_spam * P_spam) - (len(data_with_message.iloc[row, 0]) * np.log(N_spam))
            sum_spam += ans
            sum_list_spam.append(ans)
            
        ax = [sum_ham, sum_spam]
        preds[row] = np.argmax(ax)
        preds_proba.append(ax)
        print(f'{row} объект классифицирован. proba = {ax}, класс: {np.argmax(ax)}')
    return preds, preds_proba
        
        

Wall time: 0 ns


In [339]:
preds, preds_proba = predict_multinomial(test_df, test_data, thetas_dict, P_ham, P_spam, N_ham, N_spam, lambda_ham=1, lambda_spam=1)

0 объект классифицирован. proba = [-209598.18204690417, -2230629.806127515], класс: 0
1 объект классифицирован. proba = [-49740.66539859984, -2021225.031032144], класс: 0
2 объект классифицирован. proba = [-2107866.236365167, -97382.19375530128], класс: 1
3 объект классифицирован. proba = [-6131.98053780167, 35883.023546354816], класс: 1
4 объект классифицирован. proba = [-78811.29948044909, -4059290.9975162116], класс: 0
5 объект классифицирован. proba = [-5166003.572850942, -173542.5641166157], класс: 1
6 объект классифицирован. proba = [-3267767.578724363, -306822.24956560635], класс: 1
7 объект классифицирован. proba = [-7209600.609117165, -230658.16251637702], класс: 1
8 объект классифицирован. proba = [-35203.89900873026, -1002192.9117263956], класс: 0
9 объект классифицирован. proba = [-180528.52396106857, -2192560.7035778994], класс: 0
10 объект классифицирован. proba = [-136938.07266702148, -2135456.8939898955], класс: 0
11 объект классифицирован. proba = [-136951.64947111488,

95 объект классифицирован. proba = [-49736.6461480727, -1021226.5010359829], класс: 0
96 объект классифицирован. proba = [-49727.697677642944, -21226.148643133514], класс: 1
97 объект классифицирован. proba = [-93343.33908389749, -2078340.0554240765], класс: 0
98 объект классифицирован. proba = [-238708.36171017252, -7268725.928639398], класс: 0
99 объект классифицирован. proba = [-93337.40984561502, -3078332.206887987], класс: 0
100 объект классифицирован. proba = [-93346.29237794332, -3078336.6399538], класс: 0
101 объект классифицирован. proba = [-4195082.13706872, -211630.02209112988], класс: 1
102 объект классифицирован. proba = [-166031.1980746915, -7173520.905841553], класс: 0
103 объект классифицирован. proba = [-35203.97311670242, -2002186.184878617], класс: 0
104 объект классифицирован. proba = [-1195086.3263166712, -2211612.3430958344], класс: 0
105 объект классифицирован. proba = [-35201.3408589947, -2194.309485163176], класс: 1
106 объект классифицирован. proba = [-35193.3

189 объект классифицирован. proba = [-122422.98015445472, -7116400.53700204], класс: 0
190 объект классифицирован. proba = [-93342.66168507386, -3078331.3826658437], класс: 0
191 объект классифицирован. proba = [-49738.86740858087, -1021231.7578801556], класс: 0
192 объект классифицирован. proba = [-35200.55715702671, -1002187.1848786506], класс: 0
193 объект классифицирован. proba = [-93331.26426420052, -78339.86291584694], класс: 1
194 объект классифицирован. proba = [-93345.08576010757, -1078349.8666943682], класс: 0
195 объект классифицирован. proba = [-49745.51742886376, -2021227.1104738794], класс: 0
196 объект классифицирован. proba = [-2122415.281285934, -116430.42495895643], класс: 1
197 объект классифицирован. proba = [-64273.39711152522, -2040267.3429218964], класс: 0
198 объект классифицирован. proba = [-195087.428417385, -5211599.1249014605], класс: 0
199 объект классифицирован. proba = [-122421.82083661309, -5116409.650143684], класс: 0
200 объект классифицирован. proba =

283 объект классифицирован. proba = [-1180537.8288843264, -3192568.239381979], класс: 0
284 объект классифицирован. proba = [-6131.98053780167, 35883.023546354816], класс: 1
285 объект классифицирован. proba = [-93350.47364503473, -2078348.9441702298], класс: 0
286 объект классифицирован. proba = [-93340.19227358732, -78354.36843308233], класс: 1
287 объект классифицирован. proba = [-6131.98053780167, 35883.023546354816], класс: 1
288 объект классифицирован. proba = [-224166.0424418219, -8249665.427612163], класс: 0
289 объект классифицирован. proba = [-238691.83744726013, -5268719.206058945], класс: 0
290 объект классифицирован. proba = [-49735.81485055366, -21228.86266417419], класс: 1
291 объект классифицирован. proba = [-4224149.281344269, -1249685.403435892], класс: 1
292 объект классифицирован. proba = [-5136928.721941409, -135475.44734329783], класс: 1
293 объект классифицирован. proba = [-107871.59970497811, -1097387.7078673919], класс: 0
294 объект классифицирован. proba = [-7

377 объект классифицирован. proba = [-166027.3232005807, -2173539.2356482763], класс: 0
378 объект классифицирован. proba = [-49740.65916805009, -2021228.2090855564], класс: 0
379 объект классифицирован. proba = [-180554.43403167662, -2192583.0997361015], класс: 0
380 объект классифицирован. proba = [-209625.96811270044, -6230640.592405274], класс: 0
381 объект классифицирован. proba = [-1180543.2024761916, -2192557.168893614], класс: 0
382 объект классифицирован. proba = [-195090.9630855939, -4211602.805599088], класс: 0
383 объект классифицирован. proba = [-122417.41728643312, -6116405.5591008505], класс: 0
384 объект классифицирован. proba = [-78814.16611359279, -3059302.7580645364], класс: 0
385 объект классифицирован. proba = [-224142.82360855085, -3249685.904626942], класс: 0
386 объект классифицирован. proba = [-64265.16835697196, -40268.60570063611], класс: 1
387 объект классифицирован. proba = [-49735.9822551537, -2021222.5923147637], класс: 0
388 объект классифицирован. proba

...

565 объект классифицирован. proba = [-11224107.018850198, -249709.06383113103], класс: 1
566 объект классифицирован. proba = [-93344.42567241636, -2078344.3474492906], класс: 0
567 объект классифицирован. proba = [-282317.3001432771, -9325822.124058103], класс: 0
568 объект классифицирован. proba = [-166011.8158630506, -4173527.4123994363], класс: 0
569 объект классифицирован. proba = [-4253222.789244244, -287768.8418930199], класс: 1
570 объект классифицирован. proba = [-195096.27678937023, -5211607.632333974], класс: 0
571 объект классифицирован. proba = [-151478.07184512395, -3154492.5215915455], класс: 0
572 объект классифицирован. proba = [-35198.660037041824, -2194.604873621364], класс: 1
573 объект классифицирован. proba = [-122421.16116673485, -4116421.1875478756], класс: 0
574 объект классифицирован. proba = [-180568.80200481677, -5192569.072776248], класс: 0
575 объект классифицирован. proba = [-1209612.462156196, -230651.9363507422], класс: 1
576 объект классифицирован. prob

659 объект классифицирован. proba = [-180568.0347200715, -6192565.30018746], класс: 0
660 объект классифицирован. proba = [-3224164.316213676, -249696.94518841157], класс: 1
661 объект классифицирован. proba = [-64277.58474563437, -3040261.2038606834], класс: 0
662 объект классифицирован. proba = [-35201.92492770425, -2198.4599191920515], класс: 1
663 объект классифицирован. proba = [-122409.54152435514, -3116409.8583838516], класс: 0
664 объект классифицирован. proba = [-180576.1359422963, -10192540.533206016], класс: 0
665 объект классифицирован. proba = [-3209613.123858308, -230658.85094667255], класс: 1
666 объект классифицирован. proba = [-64273.447490252074, -1040274.2520910447], класс: 0
667 объект классифицирован. proba = [-93346.84183390334, -2078351.3913364175], класс: 0
668 объект классифицирован. proba = [-136949.74299014732, -5135447.803150963], класс: 0
669 объект классифицирован. proba = [-35204.969450141965, -1002191.5254319722], класс: 0
670 объект классифицирован. pro

753 объект классифицирован. proba = [-282296.5905802086, -9325814.071560826], класс: 0
754 объект классифицирован. proba = [-35202.483964955856, -1002193.6048735596], класс: 0
755 объект классифицирован. proba = [-49740.46245775585, -2021224.495514059], класс: 0
756 объект классифицирован. proba = [-35203.106231708756, -1002192.9117264205], класс: 0
757 объект классифицирован. proba = [-49741.02207354378, -1021233.0641318128], класс: 0
758 объект классифицирован. proba = [-64266.44819890153, -2040257.3933039051], класс: 0
759 объект классифицирован. proba = [-93356.5206231251, -3078342.949690333], класс: 0
760 объект классифицирован. proba = [-93332.81643214432, -1078341.914737482], класс: 0
761 объект классифицирован. proba = [-180562.37689675507, -8192548.373846356], класс: 0
762 объект классифицирован. proba = [-122401.70482250968, -3116398.810152706], класс: 0
763 объект классифицирован. proba = [-253229.49533551716, -7287749.988892379], класс: 0
764 объект классифицирован. proba =

847 объект классифицирован. proba = [-136951.7481516367, -6135442.644919351], класс: 0
848 объект классифицирован. proba = [-64272.738206872455, -1040270.0932078399], класс: 0
849 объект классифицирован. proba = [-64281.97909522217, -3040260.0407102816], класс: 0
850 объект классифицирован. proba = [-64266.53521027852, -2040257.3933038602], класс: 0
851 объект классифицирован. proba = [-64267.83883415431, -1040270.9379049473], класс: 0
852 объект классифицирован. proba = [-151489.16060331036, -7154481.407363133], класс: 0
853 объект классифицирован. proba = [-78812.43583443886, -3059300.193115193], класс: 0
854 объект классифицирован. proba = [-64274.29637495643, -2040263.8132984198], класс: 0
855 объект классифицирован. proba = [-1209611.4684040851, -1230656.6603230007], класс: 0
856 объект классифицирован. proba = [-49743.72566939453, -2021228.209085722], класс: 0
857 объект классифицирован. proba = [-107872.99254575493, -2097385.609255045], класс: 0
858 объект классифицирован. proba

941 объект классифицирован. proba = [-93339.37399985458, -2078346.501823089], класс: 0
942 объект классифицирован. proba = [-49734.882559480444, -1021228.5159390392], класс: 0
943 объект классифицирован. proba = [-354980.936956959, -7421049.369261828], класс: 0
944 объект классифицирован. proba = [-224163.98081901672, -10249652.087871784], класс: 0
945 объект классифицирован. proba = [-282323.6508130162, -11325823.365998572], класс: 0
946 объект классифицирован. proba = [-166030.9075164359, -6173531.464669579], класс: 0
947 объект классифицирован. proba = [-93334.65157451994, -78346.74062657355], класс: 1
948 объект классифицирован. proba = [-64270.40414444697, -1040270.3400677902], класс: 0
949 объект классифицирован. proba = [-64267.96370499413, -1040264.1201511871], класс: 0
950 объект классифицирован. proba = [-64271.19906424114, -1040270.6057712601], класс: 0
951 объект классифицирован. proba = [-49737.73032350072, -21236.757506860795], класс: 1
952 объект классифицирован. proba =

In [340]:
preds = pd.Series(data=preds)
preds_proba = pd.DataFrame(np.array(preds_proba))
test_data = test_data[['Message' ,'Category']]
test_data['Pred'] = preds
test_data['Proba_ham'] = preds_proba[0]
test_data['Proba_spam'] = preds_proba[1]
test_data['Pred'] = test_data['Pred'].apply(lambda x: 'ham' if (x == 0.0) else 'spam')

<p>Результаты на тестовой выборке:</p>

In [341]:
print(classification_report(test_data['Category'], test_data['Pred']))

              precision    recall  f1-score   support

         ham       0.97      0.89      0.93       867
        spam       0.53      0.85      0.66       133

    accuracy                           0.88      1000
   macro avg       0.75      0.87      0.79      1000
weighted avg       0.92      0.88      0.89      1000



<p>Результаты на тестовой выборке идентичны предыдущим</p>
<p>Посмотрим на каких объектах алгоритм ошибся:</p>

In [342]:
# Объекты, на которых алгоритм ошибся
test_data.loc[test_data['Category'] != test_data['Pred']]

,Message,Category,Pred,Proba_ham,Proba_spam
3,[],ham,spam,-6.131981e+03,3.588302e+04
62,"[wow, thats, gay, firmware, update, help]",ham,spam,-1.093337e+06,-1.078347e+06
70,[havent],ham,spam,-2.066743e+04,1.684100e+04
83,"[hope, great, day]",ham,spam,-4.973504e+04,-2.123688e+04
88,"[cant, pick, phone, right, pls, send, message]",ham,spam,-1.078751e+05,-9.739499e+04
...,...,...,...,...,...
957,[one],ham,spam,-2.066609e+04,1.684308e+04
962,"[another, number]",ham,spam,-3.520268e+04,-2.197980e+03
968,"[asked, mobile, chatlines, inclu, marketword, ...",spam,ham,-1.325894e+06,-6.382936e+06
976,"[trying, weekend]",ham,spam,-3.520380e+04,-2.197847e+03


<p>Бросается в глаза, что в большинстве случаев оценки принадлежности к классам сравнимы: они одного порядка и не сильно отличаются</p>
<h5>Отсюда следует, что объекты пограничные. Можно решить проблему, добавив в формулу разные потери от ошибок в зависимости от класса. Сейчас потери от ошибок одинаковые и равны 1</h5>

In [ ]:
preds, preds_proba = predict_multinomial(test_df, test_data, thetas_dict, P_ham, P_spam, N_ham, N_spam, lambda_ham=0.8, lambda_spam=4)

In [ ]:
preds = pd.Series(data=preds)
preds_proba = pd.DataFrame(np.array(preds_proba))
test_data = test_data[['Message' ,'Category']]
test_data['Pred'] = preds
test_data['Proba_ham'] = preds_proba[0]
test_data['Proba_spam'] = preds_proba[1]
test_data['Pred'] = test_data['Pred'].apply(lambda x: 'ham' if (x == 0.0) else 'spam')

In [ ]:
print(classification_report(test_data['Category'], test_data['Pred']))

In [ ]:
# Объекты, на которых алгоритм ошибся
test_data.loc[test_data['Category'] != test_data['Pred']]